In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
# conf.set('spark.ui.proxyBase'
# , '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/04 19:41:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [54]:
import pandas as pd
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType

schema= StructType(
      [StructField('title',StringType(),True),
      StructField('text',StringType(),True),
      StructField('subject',StringType(),True),
      StructField('date',StringType(),True)])

df0 = pd.read_csv('/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/fake.csv')
df1 = pd.read_csv('/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/Kaggle/true.csv')
df_fake = spark.createDataFrame(df0,schema=schema)
df_true = spark.createDataFrame(df1,schema=schema)

In [55]:
# df_fake.show(20)

In [56]:
# df_true.show(20)

In [57]:
# add a label to the data for fake news as 0 and true news as 1 and shuffle using rand
from pyspark.sql.functions import lit, rand

df= df_true.withColumn('flag', lit(1)).union(df_fake.withColumn('flag', lit(0))).orderBy(rand())

In [58]:
# df.show(20)

In [59]:
#types of news and highest number of news in order
from pyspark.sql.functions import col

df.groupby('subject').count().sort(col("count").desc())

22/12/04 22:05:21 WARN TaskSetManager: Stage 59 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.
22/12/04 22:05:21 WARN TaskSetManager: Stage 62 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


subject,count
politicsNews,11272
worldnews,10145
News,9050
politics,6841
left-news,4459
Government News,1570
US_News,783
Middle-east,778


In [60]:
# text pre-processing

from pyspark.sql.functions import col, split, lower, regexp_replace, length

df = df.select('title',(lower(regexp_replace('text', "[^a-zA-Z\\s]", "")).alias('text')), 'subject', 'date', 'flag')

In [61]:
# tokenize the text

from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol='text', outputCol='words_token')

df = tokenizer.transform(df).select('title','words_token','subject','date','flag')

In [62]:
# df.first()

In [63]:
# remove the stop words

# tokens=['paris', 'reuters', '', 'plans', 'to', 'buy', 'a', 'new', 'presidential', 'jet', 'for', 'france', 's', 'emmanuel', 'macron', 'could', 'be']

# clean_tokens = [ tok for tok in tokens if tok ]

# clean_tokens

In [64]:
#remove empty string from the frame
from pyspark.sql.functions import udf
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType

def space_removal(f):
    clean_tokens = []
    for tok in f:
        if tok:
            clean_tokens.append(tok)
    return clean_tokens

udf_space_removal = udf(space_removal, ArrayType(StringType()))

df = df.withColumn('words_token', udf_space_removal(f.col('words_token')))

In [65]:
# df.first()

In [66]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaakki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
# remove the stop words
from pyspark.ml.feature import StopWordsRemover

stop_words_remover = StopWordsRemover(inputCol='words_token', outputCol='text').setStopWords(stop)

In [68]:
from pyspark.ml import Pipeline

stopWordRemovalPipeline = Pipeline(stages=[stop_words_remover])
pipelineFitRemoveStopWords = stopWordRemovalPipeline.fit(df)

In [69]:
df = pipelineFitRemoveStopWords.transform(df.dropna()) #added here

In [70]:
# df.first()

In [71]:
# stematizing the words
# from nltk.stem.snowball import SnowballStemmer

# stemmer = SnowballStemmer(language='english')
# stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))

# df = df.withColumn("words_stemmed", stemmer_udf("text")).select('title','words_token','subject','date','flag','words_stemmed')

In [72]:
# df.first()

In [73]:
# df = df.withColumnRenamed("words_stemmed","text")

In [74]:
df

22/12/04 22:05:54 WARN TaskSetManager: Stage 65 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 22:05:57 WARN TaskSetManager: Stage 66 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 22:06:00 WARN TaskSetManager: Stage 69 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 22:06:03 WARN TaskSetManager: Stage 70 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


title,words_token,subject,date,flag,text
Mueller Spokesma...,"[trump, supporter...",News,"December 17, 2017",0,"[trump, supporter..."
JUST IN: FCC VOTE...,"[today, the, fcc,...",politics,"Dec 14, 2017",0,"[today, fcc, vote..."
(VIDEO) OBAMA ON ...,"[the, stats, don,...",politics,"May 5, 2015",0,"[stats, lie, one,..."
BREAKING VIDEO: T...,"[the, attack, on,...",politics,"Dec 19, 2016",0,"[attack, russian,..."
3 CONSERVATIVE CE...,"[ted, nugent, is,...",left-news,"Apr 20, 2017",0,"[ted, nugent, smi..."
BUSTED! ONE OF NA...,"[i, wonder, if, t...",politics,"Nov 2, 2016",0,"[wonder, get, tre..."
Kenya parliament ...,"[nairobi, reuters...",worldnews,"October 11, 2017",1,"[nairobi, reuters..."
BRITISH TV PERSON...,"[he, obama, makes...",politics,"Dec 9, 2015",0,"[obama, makes, wa..."
Mnangagwa vows to...,"[harare, reuters,...",worldnews,"November 23, 2017",1,"[harare, reuters,..."
Factbox - What's ...,"[madrid, reuters,...",worldnews,"December 22, 2017",1,"[madrid, reuters,..."


In [75]:
# most freq words in data

from pyspark.sql.functions import concat_ws

df_freq = df.withColumn("text", concat_ws(" ", df["text"]))

# df_freq.show(5)

In [76]:
# most frequent fake words

df_fake_freq = df_freq.filter(df["flag"] == 0)

df_fake_text = df_fake_freq.select("text")


In [77]:
from pyspark.sql.functions import explode

df_fake_text = df_fake_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_fake = df_fake_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

df_freq_fake.orderBy(col("count").desc()).show()

22/12/04 22:06:15 WARN TaskSetManager: Stage 73 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 22:06:17 WARN TaskSetManager: Stage 74 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


+---------+-----+
|     word|count|
+---------+-----+
|    trump|73933|
|     said|31013|
|   people|25963|
|president|25586|
|    would|23427|
|      one|22935|
|       us|22049|
|  clinton|18011|
|    obama|17813|
|     like|17621|
|   donald|17215|
|     also|15242|
|      new|14158|
|     news|14126|
|     even|13717|
|  hillary|13565|
|    white|12778|
|     time|12728|
|    state|12525|
|      via|11273|
+---------+-----+
only showing top 20 rows



In [26]:
#most true words

df_true_freq = df_freq.filter(df["flag"] == 1)

df_true_text = df_true_freq.select("text")

In [27]:
from pyspark.sql.functions import explode

df_true_text = df_true_text\
.withColumn("words", split(col("text"), " "))\
.where(length("text")>0)

df_freq_true = df_true_text\
.select(explode(col("words")).alias("word"))\
.groupBy("word").count()\

df_freq_true.orderBy(col("count").desc()).show()

22/12/04 19:45:21 WARN TaskSetManager: Stage 41 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:45:32 WARN TaskSetManager: Stage 42 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


+----------+-----+
|      word|count|
+----------+-----+
|      said|98985|
|     trump|54341|
|        us|41137|
|     state|36638|
|     would|31514|
|    presid|28367|
|    reuter|28306|
|republican|22108|
|    govern|20215|
|      year|19294|
|      hous|17525|
|       new|16786|
|      unit|16525|
|  democrat|16214|
|      also|15944|
|       say|15942|
|     senat|15705|
|     elect|15524|
|     peopl|15322|
|     parti|15002|
+----------+-----+
only showing top 20 rows



In [28]:
df_freq.printSchema()

root
 |-- title: string (nullable = true)
 |-- words_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- flag: integer (nullable = false)
 |-- text: string (nullable = false)



In [29]:
df_freq.count()

22/12/04 19:45:53 WARN TaskSetManager: Stage 48 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


44898

In [30]:
# drop unwanted coloumns

df_ml = df_freq.drop("title","words_token","date")
df_ml.printSchema()

root
 |-- subject: string (nullable = true)
 |-- flag: integer (nullable = false)
 |-- text: string (nullable = false)



In [31]:
# x values and Y values

x = df_ml.select('text').rdd.flatMap(lambda x: x).collect()
y = df_ml.select('flag').rdd.flatMap(lambda x: x).collect()

22/12/04 19:46:22 WARN TaskSetManager: Stage 51 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:46:25 WARN TaskSetManager: Stage 52 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:46:52 WARN TaskSetManager: Stage 55 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


22/12/04 19:46:56 WARN TaskSetManager: Stage 56 contains a task of very large size (4983 KiB). The maximum recommended task size is 1000 KiB.


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.25)

In [33]:
print(x_train[0])
print(y_train[0])

peopl petit start juli th ask govern formal design black live matter terrorist organ white hous requir person group initi petit peopl petit page must gather least signatur day signatur white hous site guarante review petit offer offici respons obama doubt spin meantim statement american citizen belief black live matter noth terror organ overwhelm support label black live matter terror group bode well hillari open support embarrass effort pander votesfrom white hous creat peopl petit pageabout peopleth right petit govern guarante first amend unit state constitut peopl platform empow american public take action like never way anybodi anywher speak direct govern becom agent changewith peopl easili creat petit onlin share collect signatur gather signatur day review petit make sure get front appropri polici expert issu offici responsepetit potenti enact real chang check petit creator success stori also fundament right american citizen opportun connect communiti likemind peopl invest make ch

In [34]:
#vectorize the text

from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [36]:
xv_train

<33673x157070 sparse matrix of type '<class 'numpy.float64'>'
	with 5098041 stored elements in Compressed Sparse Row format>

In [37]:
xv_train.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [40]:
LR.score(xv_test, y_test)

0.986369710467706

In [53]:
pred_LR = LR.predict(xv_test)

In [42]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5877
           1       0.98      0.99      0.99      5348

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [43]:
import re, string
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [48]:
import pandas as pd
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    print("*****")
    #print(pred_LR[0])
    print(new_xv_test)
    print(type(pred_LR[0]))
    # pred_DT = DT.predict(new_xv_test)
    # pred_GBC = GBC.predict(new_xv_test)
    # pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \n".format(output_lable(pred_LR[0])))

In [49]:
news = str(input())
manual_testing(news)

 The following statements were posted to the verified Twitter accounts of U.S. President Donald Trump, @realDonaldTrump and @POTUS.  The opinions expressed are his own. Reuters has not edited the statements or confirmed their accuracy.  @realDonaldTrump : - Vanity Fair, which looks like it is on its last legs, is bending over backwards in apologizing for the minor hit they took at Crooked H. Anna Wintour, who was all set to be Amb to Court of St James’s & a big fundraiser for CH, is beside herself in grief & begging for forgiveness! [1024 EST] -- Source link: (bit.ly/2jBh4LU) (bit.ly/2jpEXYR)


*****
0
  (0, 152967)	0.16096178767446143
  (0, 143780)	0.054190873866555564
  (0, 142249)	0.02866970596549606
  (0, 140603)	0.04939542872986928
  (0, 140058)	0.5176326165631658
  (0, 130228)	0.06689479292151192
  (0, 124619)	0.052321727153028616
  (0, 113471)	0.14800674609052442
  (0, 108409)	0.1725442055210553
  (0, 107617)	0.09057757025271987
  (0, 97681)	0.08030345818911086
  (0, 97350)	0.11235736170548934
  (0, 96237)	0.1677370677824023
  (0, 95306)	0.16400836207085898
  (0, 94316)	0.32192357534892285
  (0, 92587)	0.1793194856289962
  (0, 85044)	0.06598028632480062
  (0, 79234)	0.32801672414171795
  (0, 77282)	0.0654216862280709
  (0, 77004)	0.033483274037078736
  (0, 74638)	0.035840970070839646
  (0, 67735)	0.1793194856289962
  (0, 64352)	0.3048518884485302
  (0, 58713)	0.06287071507041829
  (0, 53124)	0.11647021989379128
  (0, 44061)	0.06961458558868093
  (0, 36815)	0.0317254189439036
  (0, 30271)	0.1677370677824023
  (0, 29327)	0.05127258504891143
  (0, 21784)	0.140843526377671